In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.15.0


In [29]:
model = keras.models.Sequential([keras.layers.Input(shape=(7,)),
                                 keras.layers.Dense(300, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(1, activation="relu", use_bias=False)])

model.compile(loss="mean_squared_error", optimizer="adam")

In [4]:
def generate_random_data(n_samples=1000):
    # Generate random features
    X = np.random.normal(size=(n_samples, 7))

    # Generate weights for linear combination
    weights = np.random.uniform(-1, 1, size=7)

    # Generate target variable as a linear combination of features plus noise
    noise = np.random.normal(scale=0.5, size=n_samples)
    y = X @ weights + noise

    return X, y

In [5]:
X_train, y_train = generate_random_data(n_samples=1000)
init_w = model.get_weights()
model.fit(X_train, y_train, epochs=10, batch_size=32)
w = model.get_weights()

Epoch 1/10

32/32 [==============================] - 2s 4ms/step - loss: 1.5969
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 1.3747
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 1.3704
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 1.3503
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 1.3601
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 1.3573
Epoch 7/10
32/32 [==============================] - 0s 4ms/step - loss: 1.3420
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 1.3342
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 1.3369
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 1.3323


In [6]:
weights = model.get_weights()
initial_weights = [tf.constant(w, dtype=tf.float32) for w in weights]

In [12]:
def gpt_l1_projection(w, gamma):
    # Test if projection necessary
    if tf.norm(w, ord=1) <= gamma:
        return w
    else:
        # Apply L1-projection on weight vector w
        w_shape = tf.shape(w)
        w_flat = tf.reshape(w, [-1])
        abs_w_flat = tf.abs(w_flat)

        # Compute cumulative sum of the sorted absolute weights
        u = tf.sort(abs_w_flat, direction="DESCENDING")
        svp = tf.cumsum(u)

        # Find the position where the condition is violated for the first time
        rho = tf.range(1, tf.size(u, out_type=tf.int32) + 1, dtype=tf.float32)
        cond = (svp - gamma) / rho
        mask = cond > 0
        # Ensure at least one True exists to avoid errors with argmax.
        if not tf.reduce_any(mask):
            return w
        k = tf.argmax(tf.cast(mask, tf.int32), output_type=tf.int32)

        # Compute the threshold value
        theta = (tf.gather(svp, k) - gamma) / tf.cast(k + 1, tf.float32)

        # Apply the thresholding operation
        projected_weights_flat = tf.sign(w_flat) * tf.maximum(abs_w_flat - theta, 0.0)
        return tf.reshape(projected_weights_flat, w_shape)


In [2]:
@tf.function
def apply_l1_projection(w, gamma):
        # Test if projection necessary
        if tf.norm(w, ord=1) <= gamma:
            return w
        # Apply L1-projection on weight vector w
        abs_w = tf.abs(w)

        # Compute cumulative sum of the sorted absolute weights
        u = tf.sort(abs_w, direction="DESCENDING")
        svp = tf.cumsum(u)

        # Find the position where the condition is violated for the first time
        cond = (svp - gamma) / tf.range(1, tf.size(u) + 1, dtype=tf.float64)
        k = tf.reduce_max(tf.where(u > cond))

        # Compute the threshold value
        theta = (tf.gather(svp,k) - gamma)/ tf.cast(k + 1, tf.float64)

        # Apply the thresholding operation
        
        return tf.math.sign(w) * tf.maximum(abs_w - theta, 0)

In [11]:
w = np.random.uniform(-1,1,500)

In [15]:
abs_w = tf.abs(w)
gamma = 1.0

# Compute cumulative sum of the sorted absolute weights
u = tf.sort(abs_w, direction="DESCENDING")
svp = tf.cumsum(u)
# Find the position where the condition is violated for the first time
cond = (svp - gamma) / tf.range(1, tf.size(u) + 1, dtype=tf.float64)
k = tf.reduce_max(tf.where(u > cond))

# Compute the threshold value
theta = (tf.gather(svp,k) - gamma)/ tf.cast(k + 1, tf.float64)

print("Theta : ", theta)
print("K : ",k)

Theta :  tf.Tensor(0.9327760751130038, shape=(), dtype=float64)
K :  tf.Tensor(31, shape=(), dtype=int64)


In [17]:
abs_w = tf.abs(w)
gamma = 1.0

# Compute cumulative sum of the sorted absolute weights
u = tf.sort(abs_w, direction="DESCENDING")
svp = tf.cumsum(u)
# Find the position where the condition is violated for the first time
cond = (svp - gamma) / tf.range(1, tf.size(u) + 1, dtype=tf.float64)
rho = tf.math.count_nonzero(u > cond)

# Compute the threshold value
theta = tf.cast(tf.gather(svp, rho - 1) - gamma, tf.float32) / tf.cast(
        rho, tf.float32
    )

print("Theta : ", theta)
print("K : ",k)

Theta :  tf.Tensor(0.9327761, shape=(), dtype=float32)
K :  tf.Tensor(31, shape=(), dtype=int64)


In [49]:
x = model.trainable_variables[8]
y = tf.sort(x, direction="DESCENDING", axis=0)
tf.reduce_sum(tf.where(y > 0.0, tf.ones_like(x), tf.zeros_like(x)),axis=0)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([56.], dtype=float32)>

In [95]:
def function(w, gamma):
    if tf.norm(w, ord=1) <= gamma:
        return w
    # Apply L1-projection on weight vector w
    abs_w = tf.abs(w)

    # Compute cumulative sum of the sorted absolute weights
    u = tf.sort(abs_w, direction="DESCENDING", axis=0)
    svp = tf.cast(tf.cumsum(u, axis=0) - gamma, tf.float64)

    # Find the position where the condition is violated for the first time
    ratio = tf.cast(svp / tf.range(1, tf.size(u) + 1, dtype=tf.float64),dtype=tf.float32)

    # Compute the threshold value
    theta = tf.reduce_max(tf.maximum(ratio, 0.0), axis=0)

    return tf.math.sign(w) * tf.maximum(abs_w - theta, 0)

In [97]:
x = tf.constant([0,2.7,2.0,2.0,5.,1,7.,10,5 ])
tf.norm(x, ord=1)

<tf.Tensor: shape=(), dtype=float32, numpy=34.7>

In [101]:
def function(w, gamma):
    if tf.norm(w, ord=1) <= gamma:
        return w
    # Apply L1-projection on weight vector w
    abs_w = tf.abs(w)

    # Compute cumulative sum of the sorted absolute weights
    u = tf.sort(abs_w, direction="DESCENDING", axis=0)
    svp = tf.cumsum(u, axis=0) - gamma

    # Find the position where the condition is violated for the first time
    ratio = svp / tf.range(1, tf.size(u) + 1, dtype=u.dtype)

    # Compute the threshold value
    theta = tf.reduce_max(tf.maximum(ratio, 0.0), axis=0)

    return tf.math.sign(w) * tf.maximum(abs_w - theta, 0)

import inspect

inspect.findsource(function)

(['def function(w, gamma):\n',
  '    if tf.norm(w, ord=1) <= gamma:\n',
  '        return w\n',
  '    # Apply L1-projection on weight vector w\n',
  '    abs_w = tf.abs(w)\n',
  '\n',
  '    # Compute cumulative sum of the sorted absolute weights\n',
  '    u = tf.sort(abs_w, direction="DESCENDING", axis=0)\n',
  '    svp = tf.cumsum(u, axis=0) - gamma\n',
  '\n',
  '    # Find the position where the condition is violated for the first time\n',
  '    ratio = svp / tf.range(1, tf.size(u) + 1, dtype=u.dtype)\n',
  '\n',
  '    # Compute the threshold value\n',
  '    theta = tf.reduce_max(tf.maximum(ratio, 0.0), axis=0)\n',
  '\n',
  '    return tf.math.sign(w) * tf.maximum(abs_w - theta, 0)\n',
  '\n',
  'import inspect\n',
  '\n',
  'inspect.findsource(function)\n'],
 0)

In [100]:
x = 1
y = tf.constant([x])
print(y)

tf.Tensor([1], shape=(1,), dtype=int32)


In [98]:
y = apply_l1(x, 10.0)
print(y)
print(tf.norm(y, ord=1))

tf.Tensor(
[0.        3.5       4.        4.25      3.94      3.6166668 3.3857143
 3.0875    2.7444446], shape=(9,), dtype=float32)
tf.Tensor(4.25, shape=(), dtype=float32)
tf.Tensor([10.   7.   5.   5.   2.7  2.   2.   1.   0. ], shape=(9,), dtype=float32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4.25, shape=(), dtype=float32)
tf.Tensor([0.   0.   0.   0.   0.75 0.   2.75 5.75 0.75], shape=(9,), dtype=float32)
tf.Tensor(10.0, shape=(), dtype=float32)


In [3]:
def l1_projection_sparse(w, gamma):
    # Test if projection necessary
    if tf.norm(w, ord=1) <= gamma:
        return w
    
    perturbed_indices = tf.where(tf.abs(w) > 0)
    
    # Project only the perturbed elements
    perturbed_w = w[perturbed_indices]
    projected_perturbed_w = apply_l1_projection(perturbed_w, gamma)
    
    # Reconstruct the full vector
    w[perturbed_indices] = projected_perturbed_w
    
    return w

In [4]:
import time
import numpy as np

In [6]:
times = []
for i in range(1000):
    # print("--------------------")
    # print("Round : ", i)
    w = np.random.uniform(-1,1,500)
    print("Norm before projection : ", np.linalg.norm(w,1))
    start_time = time.time()
    v = apply_l1_projection(w,10)
    t = time.time() - start_time
    times.append(t)
    # print("Time : ", t)
    print("Norm after projection: ", np.linalg.norm(v,1))

print("Mean time : ", np.mean(times))

Norm before projection :  263.2824653174285
Norm after projection:  9.999999999999972
Norm before projection :  246.54843507194676
Norm after projection:  9.999999999999929
Norm before projection :  240.91160117846215
Norm after projection:  10.000000000000036
Norm before projection :  243.9881417148088
Norm after projection:  9.999999999999943
Norm before projection :  251.9125901927057
Norm after projection:  10.000000000000039
Norm before projection :  230.7319754984275
Norm after projection:  9.999999999999952
Norm before projection :  244.52931661417858
Norm after projection:  9.999999999999975
Norm before projection :  255.77872976727457
Norm after projection:  9.99999999999995
Norm before projection :  247.90085032400648
Norm after projection:  10.000000000000012
Norm before projection :  255.76179813931273
Norm after projection:  9.99999999999999
Norm before projection :  241.37814479264003
Norm after projection:  10.000000000000014
Norm before projection :  250.98179882567595


In [7]:
times = []
for i in range(1000):
    # print("--------------------")
    # print("Round : ", i)
    w = np.random.uniform(-1,1,500)
    print("Norm before projection : ", np.linalg.norm(w,1))
    start_time = time.time()
    v = l1_projection_sparse(w,10)
    t = time.time() - start_time
    times.append(t)
    # print("Time : ", t)
    print("Norm after projection: ", np.linalg.norm(v,1))

print("Mean time : ", np.mean(times))

Norm before projection :  257.4107037043209
Norm after projection:  70.57917355120665
Norm before projection :  254.29858502161355
Norm after projection:  66.05274625389511
Norm before projection :  239.83306598188392
Norm after projection:  64.88365590410305
Norm before projection :  251.1015864495334
Norm after projection:  67.21714091698954
Norm before projection :  255.9090660931808
Norm after projection:  66.64261311002315
Norm before projection :  241.50216393046856
Norm after projection:  67.14874327192872
Norm before projection :  246.66170678876745
Norm after projection:  68.15492033631315
Norm before projection :  250.87630077129754
Norm after projection:  68.81308597939915
Norm before projection :  253.65941602872346
Norm after projection:  70.26129216566672
Norm before projection :  241.4290665365591
Norm after projection:  67.50483997673982
Norm before projection :  255.3439673860079
Norm after projection:  68.07922547866477
Norm before projection :  253.87234214307452
Nor

In [ ]:
Mean time :  0.10532336235046387
Mean time :  0.08903295993804931

In [ ]:
def apply_l2_projection(delta, init_w, weights):
    weights_diff = tf.concat([w - w0 for w, w0 in zip(weights, initial_weights)], axis=0)
    projected_weights = init_w + tf.clip_by_norm(
        weights_diff, delta
    )

    start = 0
    for w in weights:
        shape = w.shape
        size = tf.reduce_prod(shape)
        w.assign(tf.reshape(projected_weights[start : start + size], shape))
        start += size

In [101]:
flat_init_w = tf.concat([tf.reshape(w, [-1]) for w in initial_weights], axis=0)
print(tf.norm(flat_init_w,2))

tf.Tensor(13.537726, shape=(), dtype=float32)


In [102]:
flat_w = tf.concat([tf.reshape(x, [-1]) for x in w], axis=0)
print(tf.norm(flat_w,2))

tf.Tensor(19.109417, shape=(), dtype=float32)


In [106]:
test = tf.constant([[1,2,3,4],[1,2,3,4],[1,2,3,4]], dtype=tf.float32)
test_flat = tf.reshape(test, [-1])

print(tf.norm(test_flat,2))
print(tf.norm(test,2))

clipped = tf.clip_by_norm(test_flat, 5)
clipped_test = tf.clip_by_norm(test, 5)

print(tf.norm(clipped,2))
print(tf.norm(clipped_test,2))

print(clipped)
print(clipped_test)

tf.Tensor(9.486833, shape=(), dtype=float32)
tf.Tensor(9.486833, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(
[0.5270463 1.0540926 1.5811388 2.1081853 0.5270463 1.0540926 1.5811388
 2.1081853 0.5270463 1.0540926 1.5811388 2.1081853], shape=(12,), dtype=float32)
tf.Tensor(
[[0.5270463 1.0540926 1.5811388 2.1081853]
 [0.5270463 1.0540926 1.5811388 2.1081853]
 [0.5270463 1.0540926 1.5811388 2.1081853]], shape=(3, 4), dtype=float32)


In [120]:
print(tf.concat([test,clipped_test],axis=0))
print(tf.norm(tf.concat([test,clipped_test],axis=0),2))

print(tf.concat([test_flat,clipped],axis=0))
print(tf.norm(tf.concat([test_flat,clipped],axis=0),2))

tf.Tensor(
[[1.        2.        3.        4.       ]
 [1.        2.        3.        4.       ]
 [1.        2.        3.        4.       ]
 [0.5270463 1.0540926 1.5811388 2.1081853]
 [0.5270463 1.0540926 1.5811388 2.1081853]
 [0.5270463 1.0540926 1.5811388 2.1081853]], shape=(6, 4), dtype=float32)
tf.Tensor(10.723805, shape=(), dtype=float32)
tf.Tensor(
[1.        2.        3.        4.        1.        2.        3.
 4.        1.        2.        3.        4.        0.5270463 1.0540926
 1.5811388 2.1081853 0.5270463 1.0540926 1.5811388 2.1081853 0.5270463
 1.0540926 1.5811388 2.1081853], shape=(24,), dtype=float32)
tf.Tensor(10.723805, shape=(), dtype=float32)


In [135]:
def l2_projection_v1(delta, init_vars, vars):
    vars_diff = tf.concat([tf.reshape(v - v0,[-1]) for v, v0 in zip(vars, init_vars)],axis=0)

    if tf.norm(vars_diff,2) <= delta:
        return vars
    else:
        init_vars_concat = tf.concat([tf.reshape(v0,-1) for v0 in init_vars], axis=0)
        projected_vars = init_vars_concat + tf.clip_by_norm(vars_diff, delta)

    start = 0
    for v in vars:
        size = tf.reduce_prod(v.shape)
        v.assign(tf.reshape(projected_vars[start : start + size], v.shape))
        start += size
    return vars

In [187]:
# no reshaping of any weights
# 

def l2_projection_global(delta, init_vars, vars):
    vars_diff = [v - v0 for v, v0 in zip(vars, init_vars)]

    if tf.linalg.global_norm(vars_diff,2) > delta:
        projected_diff = tf.clip_by_global_norm(vars_diff, delta)[0]
        projected_vars = [v0 + pv for v0, pv in zip(init_vars, projected_diff)]
        return projected_vars

In [191]:
model = keras.models.Sequential([keras.layers.Input(shape=(7,)),
                                 keras.layers.Dense(300, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(1, activation="relu")])

model.compile(loss="mean_squared_error", optimizer="adam")

times = []
init_vars = [tf.identity(v) for v in model.trainable_variables]
for i in range(10):
    print("--------------------")
    print("Round : ", i)
    X_train, y_train = generate_random_data(n_samples=1000)
    model.fit(X_train, y_train, epochs=10, batch_size=32)
    vars = model.trainable_variables
    vars_diff = tf.concat([tf.reshape(v - v0,[-1]) for v, v0 in zip(vars, init_vars)],axis=0)
    print("Norm before projection : ", tf.norm(vars_diff,2))
    start_time = time.time()
    v = l2_projection_v1(0.5, init_vars, vars)
    t = time.time() - start_time
    times.append(t)
    print("Time : ", t)
    v_norm = tf.concat([tf.reshape(x - v0,-1) for x, v0 in zip(v, init_vars)], axis=0)
    print("Norm after projection: ", tf.norm(v_norm,2))
print("Mean time : ", np.mean(times))

--------------------
Round :  0
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.9593
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7249
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7253
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6971
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6559
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6552
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7379
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6822
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.7071
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6246
Norm before projection :  tf.Tensor(4.6311197, shape=(), dtype=float32)
Time :  0.13776803016662598
Norm after projection:  tf.Tensor(0.5, shape=(), dtype=float32)
--------------------
Round :  1
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.2616
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss

In [192]:
model = keras.models.Sequential([keras.layers.Input(shape=(7,)),
                                 keras.layers.Dense(300, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(100, activation="relu"),
                                 keras.layers.Dense(1, activation="relu")])

model.compile(loss="mean_squared_error", optimizer="adam")

times = []
init_vars = [tf.identity(v) for v in model.trainable_variables]
for i in range(10):
    print("--------------------")
    print("Round : ", i)
    X_train, y_train = generate_random_data(n_samples=1000)
    model.fit(X_train, y_train, epochs=10, batch_size=32)
    vars = model.trainable_variables
    vars_diff_conc = tf.concat([tf.reshape(v - v0,[-1]) for v, v0 in zip(vars, init_vars)],axis=0)
    vars_diff = [v - v0 for v, v0 in zip(vars, init_vars)]
    print("Norm before projection : ", tf.norm(vars_diff_conc,2))
    print("Global norm before projection : ", tf.linalg.global_norm(vars_diff,2))
    start_time = time.time()
    v = l2_projection_global(0.5, init_vars, vars)
    t = time.time() - start_time
    times.append(t)
    print("Time : ", t)
    v_norm_global = [x - v0 for x, v0 in zip(v, init_vars)]
    v_norm = tf.concat([tf.reshape(x - v0,-1) for x, v0 in zip(v, init_vars)], axis=0)
    print("global Norm after projection: ", tf.linalg.global_norm(v_norm_global,2))
    print("Norm after projection: ", tf.norm(v_norm,2))
print("Mean time : ", np.mean(times))

--------------------
Round :  0
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 2.5508
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.8007
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4401
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.6217
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5458
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5310
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5995
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.7099
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.6877
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6648
Norm before projection :  tf.Tensor(4.016055, shape=(), dtype=float32)
Global norm before projection :  tf.Tensor(4.016055, shape=(), dtype=float32)
Time :  0.03668522834777832
global Norm after projection:  tf.Tensor(0.5, shape=(), dtype=float32)
Norm after projection:  tf.Tensor(0.5, shape=(), dtype=float32)
-----

In [171]:
times_x = []
for i in range(100):
    print("--------------------")
    print("Round : ", i)
    start_time = time.time()
    diff = tf.concat([tf.reshape(v - v0,[-1]) for v, v0 in zip(vars, init_vars)],axis=0)
    norm = tf.norm(diff,2)
    print("Time : ", time.time() - start_time)
    print(norm)
    times_x.append(time.time() - start_time)

print("Mean time : ", np.mean(times_x))


--------------------
Round :  0
Time :  0.018146991729736328
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  1
Time :  0.023099660873413086
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  2
Time :  0.012262105941772461
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  3
Time :  0.01388406753540039
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  4
Time :  0.013963937759399414
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  5
Time :  0.025944232940673828
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  6
Time :  0.043424129486083984
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  7
Time :  0.01746845245361328
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  8
Time :  0.029920101165771484
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  

In [172]:
times_x = []
for i in range(100):
    print("--------------------")
    print("Round : ", i)
    init_reshaped = tf.concat([tf.reshape(v,[-1]) for v in init_vars],axis=0)
    start_time = time.time()
    vars_reshaped = tf.concat([tf.reshape(v,[-1]) for v in vars],axis=0)
    diff = vars_reshaped - init_reshaped
    norm = tf.norm(diff,2)
    print("Time : ", time.time() - start_time)
    print(norm)
    times_x.append(time.time() - start_time)

print("Mean time : ", np.mean(times_x))

--------------------
Round :  0
Time :  0.010963916778564453
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  1
Time :  0.010118722915649414
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  2
Time :  0.011454105377197266
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  3
Time :  0.024811506271362305
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  4
Time :  0.019989967346191406
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  5
Time :  0.011965036392211914
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  6
Time :  0.01910686492919922
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  7
Time :  0.010971784591674805
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  8
Time :  0.03305554389953613
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  

In [175]:
times_x = []
for i in range(100):
    print("--------------------")
    print("Round : ", i)
    start_time = time.time()
    diff = [vars - init for vars, init in zip(vars, init_vars)]
    norm = tf.sqrt(tf.reduce_sum([tf.norm(t,2)**2 for t in diff]))
    print("Time : ", time.time() - start_time)
    print(norm)
    times_x.append(time.time() - start_time)

print("Mean time : ", np.mean(times_x))

--------------------
Round :  0
Time :  0.12935900688171387
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  1
Time :  0.029918432235717773
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  2
Time :  0.04608154296875
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  3
Time :  0.04288649559020996
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  4
Time :  0.0243527889251709
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  5
Time :  0.030921220779418945
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  6
Time :  0.02651357650756836
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  7
Time :  0.04502534866333008
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  8
Time :  0.03006291389465332
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  9
Time : 

In [176]:
times_x = []
for i in range(100):
    print("--------------------")
    print("Round : ", i)
    start_time = time.time()
    diff = [vars - init for vars, init in zip(vars, init_vars)]
    norm = tf.linalg.global_norm(diff)
    print("Time : ", time.time() - start_time)
    print(norm)
    times_x.append(time.time() - start_time)

print("Mean time : ", np.mean(times_x))

--------------------
Round :  0
Time :  0.1081697940826416
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  1
Time :  0.009972333908081055
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  2
Time :  0.004988193511962891
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  3
Time :  0.01096963882446289
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  4
Time :  0.00797891616821289
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  5
Time :  0.01119852066040039
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  6
Time :  0.015115022659301758
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  7
Time :  0.004985332489013672
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  8
Time :  0.011845111846923828
tf.Tensor(0.49999997, shape=(), dtype=float32)
--------------------
Round :  9
T

In [24]:
global_clipped = tf.clip_by_global_norm(vars, 0.5)
print(tf.linalg.global_norm(global_clipped[0]))
print(len(global_clipped[0])) 

TypeError: `t_list` should be a sequence of tensors. Received <class 'builtin_function_or_method'>.

In [25]:
from src.my_dnn import create_dnn, create_sub_network, TruncateLayer, L1Projection
from data.data_generator import get_data, preprocess
import time

In [26]:
def train_and_evaluate_dnn(model, train_data, test_data, epochs=75):
    """
    Trains the model on the given data and evaluates its performance.
    """
    model.fit(train_data, epochs=epochs, verbose=0)
    mse, mae = model.evaluate(test_data, verbose=0)
    return mse, mae

In [27]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

In [8]:
mses = []  # Initialize empty list to store MSEs
maes = []  # Initialize empty list to store MAEs
times_dnn = []  # Initialize empty list to store times
for _ in range(10):
    x_train, y_train = get_data(
        regression_func, x_dim=input_dim, num_samples=100, sigma=0.05
    )
    x_test, y_test = get_data(
        regression_func, x_dim=input_dim, num_samples=10**5, sigma=0.05
    )

    # Preprocess data
    train_data = preprocess(x_train, y_train, batch_size=100, training=True)
    test_data = preprocess(x_test, y_test, batch_size=100, training=False)

    start_time = time.time()
    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=10,
        num_layers=10,
        num_neurons=20,
        beta=80,
        gamma=80,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=np.exp(-15))

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )
    mse, mae = train_and_evaluate_dnn(model, train_data, test_data, epochs=500)
    times_dnn.append(time.time() - start_time)
    mses.append(mse)
    maes.append(mae)
print("Time : ", np.mean(times_dnn))




Time :  59.59920418262482


In [28]:
class L2GlobalProjection(keras.Model):
    """
    A Keras model that performs an L2-projection of the weights in every
    training step, to ensure that the eucledian distance between the weights and
    the initial weights is smaller or equal to 'delta'.

    Parameters
    ----------
    delta: float
        The value to specify radius of the L2-ball around 'sub_nets_init_weights'.
        The weights will be projected on the feasible set where the L2-norm of
        the difference between current weight vector and the initial weight
        vector is smaller or equal to delta.

    """

    def __init__(
        self,
        delta,
        **kwargs
    ):
        super(L2GlobalProjection, self).__init__(**kwargs)
        self.delta = tf.constant(delta, dtype=tf.float32)
        self.init_vars = None

        # Error handling:
        if self.delta is None:
            raise ValueError("Missing required argument delta.")


    @tf.function(reduce_retracing=True)
    def train_step(self, data):
        """
        Custom training step for the model.

        After applying the gradients, the weight vector will be projected on the
        L2-ball with radius 'delta' around the center 'sub_nets_init_weights'.

        Parameters
        ----------
        data: float
            data used for training

        """

        # Unpack the data used for training
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            # Compute the loss value
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Apply L2 projection
        vars = self.trainable_variables[:-1]
        vars_diff = [v - v0 for v, v0 in zip(vars, self.init_vars)]

        if tf.linalg.global_norm(vars_diff) > self.delta:
            # Project the weights and reshape back
            projected_vars = self.l2_projection_global(vars_diff)
            for v, pv in zip(vars, projected_vars):
                v.assign(pv)

        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def l2_projection_global(self, vars_diff):
        projected_diff = tf.clip_by_global_norm(vars_diff, self.delta)[0]
        projected_vars = [v0 + pv for v0, pv in zip(self.init_vars, projected_diff)]
        return projected_vars

    def get_config(self):
        return {
            "delta": self.delta
        }

In [29]:
from keras.layers import Layer, Input, Dense, Concatenate
from keras.initializers import RandomUniform, Zeros

In [30]:
def create_global_dnn(
    train_shape,
    num_networks=None,
    num_layers=None,
    num_neurons=5,
    beta=None,
    gamma=None,
    delta=1,
):
    """
    Creates a model with num_networks subnetworks with num_layers hidden layers
    each. The output is the average of the outputs of the subnetworks.

    Parameters
    ----------
    train_shape: tuple
        shape of the training data
    num_networks: int, optional
        number of subnetworks to train (default: math.ceil(n**((np.log(n)**1.1 + 1))))
    num_layers: int, optional
        number of hidden layers in each subnetwork (default: math.ceil(np.log(n)))
    num_neurons: int, optional
        number of neurons in each hidden layer (default: 5)
    beta: float, optional
        parameter for the truncation layer (default: 10*np.log(n))
    gamma: float, optional
        parameter for the L1 projection layer (default: 10*n**(d/(2*(1+d))))
    delta: float, optional
        parameter for the L2 projection (default: 1)

    """
    # Define input shape based on dimension of input variable
    n, d = train_shape
    input_shape = (d,)

    # Create a list containing num_networks DNNs with num_layers hidden layers
    sub_networks = [
        create_sub_network(
            n=n, num_neurons=num_neurons, num_layers=num_layers, beta=beta
        )
        for _ in range(num_networks)
    ]

    # Create the output layer
    output_layer = Dense(
        units=1,
        use_bias=False,
        kernel_initializer=Zeros(),
        kernel_constraint=L1Projection(gamma),
    )

    # Define the structure of the combined model
    inputs = Input(shape=input_shape)
    truncated_outputs = [sub_net(inputs) for sub_net in sub_networks]
    concatenated_outputs = Concatenate()(truncated_outputs)
    outputs = output_layer(concatenated_outputs)

    # Create the model
    model = L2GlobalProjection(
        inputs=inputs,
        outputs=outputs,
        delta=delta,
    )

    # One-time initialization of initial weights
    if model.init_vars is None:
        init_vars = [tf.identity(v) for v in model.trainable_variables[:-1]]
        model.init_vars = init_vars

    return model

In [31]:
mses = []  # Initialize empty list to store MSEs
maes = []  # Initialize empty list to store MAEs
times_dnn = []  # Initialize empty list to store times
for _ in range(10):
    x_train, y_train = get_data(
        regression_func, x_dim=input_dim, num_samples=100, sigma=0.05
    )
    x_test, y_test = get_data(
        regression_func, x_dim=input_dim, num_samples=10**5, sigma=0.05
    )

    # Preprocess data
    train_data = preprocess(x_train, y_train, batch_size=100, training=True)
    test_data = preprocess(x_test, y_test, batch_size=100, training=False)

    start_time = time.time()
    model = create_global_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=10,
        num_layers=10,
        num_neurons=20,
        beta=80,
        gamma=80,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=np.exp(-15))

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )
    mse, mae = train_and_evaluate_dnn(model, train_data, test_data, epochs=500)
    times_dnn.append(time.time() - start_time)
    mses.append(mse)
    maes.append(mae)
print("Time : ", np.mean(times_dnn))


Time :  34.99804162979126


In [32]:
print(mses, maes)

[1.77312171459198, 1.4976550340652466, 1.4766840934753418, 1.5775160789489746, 1.5789949893951416, 1.5599907636642456, 1.3772720098495483, 2.1619603633880615, 1.5018829107284546, 1.5277957916259766] [1.77312171459198, 1.4976550340652466, 1.4766840934753418, 1.5775160789489746, 1.5789949893951416, 1.5599907636642456, 1.3772720098495483, 2.1619603633880615, 1.5018829107284546, 1.5277957916259766]


In [48]:
def test_norm_l1(model):
    # Test if L1 projection of last layer worked
    weights = tf.reshape(model.trainable_variables[-1], [-1])
    norm = tf.norm(weights, ord=1)
    print(f"norm: {norm}, gamma: 80")


def test_norm_l2_global(model):
    # Test if L2 projection of inner weights worked
    diff = [v - v0 for v, v0 in zip(model.trainable_variables[:-1], model.init_vars)]
    norm = tf.linalg.global_norm(diff)
    print(f"norm: {norm}, delta: {model.delta}")

def test_norm_l2(model):
    # Test if L2 projection of inner weights worked
    current_weights = tf.concat(
        [tf.reshape(v, [-1]) for v in model.trainable_variables[:-1]], axis=0
    )
    sub_nets_init_weights = tf.concat(
        [tf.reshape(v, [-1]) for v in model.init_vars], axis=0
    )
    diff = sub_nets_init_weights - current_weights
    norm = tf.norm(diff)
    print("norm: {}, delta: {}".format(norm, model.delta))

In [49]:
test_norm_l1(model)
test_norm_l2(model)
test_norm_l2_global(model)

norm: 0.06794852763414383, gamma: 80
norm: 0.00032215382088907063, delta: 1.0
norm: 0.0003221537917852402, delta: 1.0
